# User's Guide, Chapter 53: Advanced Corpus and Metadata Searching

We saw in :ref:`Chapter 11<usersGuide_11_corpusSearching>` some ways to work with and search through the "core" corpus.  Not everything is in the core corpus, of course, so the `converter.parse()` function is a great way of getting files from a local hard drive or the internet.  But the "core" corpus also has many great search functions, and these can be helpful for working with your own files and files on the web as well.

In this chapter, we'll introduce the other "Corpora" in addition to the "core" corpus and how they might be used.

## The Default Local Corpus

In [1]:
localCorpus = corpus.corpora.LocalCorpus()
localCorpus

<music21.corpus.corpora.LocalCorpus: 'local'>

You can add and remove paths from a *local* corpus with the ``addPath()`` and
``removePath()`` methods:

## Creating multiple corpus repositories via local corpora

In addition to the default local corpus, music21 allows users to create
and save as many named local corpora as they like, which will persist from
session to session.

Let's create a new *local* corpus, give it a directory to find music files in,
and then save it:


In [2]:
from music21 import *

aNewLocalCorpus = corpus.corpora.LocalCorpus('newCorpus')
aNewLocalCorpus.existsInSettings

False

In [3]:
aNewLocalCorpus.addPath('~/Desktop')
#_DOCS_SHOW aNewLocalCorpus.directoryPaths
print("('/Users/josiah/Desktop',)") #_DOCS_HIDE

('/Users/josiah/Desktop',)


In [4]:
aNewLocalCorpus.save()
aNewLocalCorpus.existsInSettings

/Users/cuthbert/git/music21base/music21/corpus/corpora.py: WARNING: newCorpus metadata cache: starting processing of paths: 1
/Users/cuthbert/git/music21base/music21/corpus/corpora.py: WARNING: cache: filename: /var/folders/1t/0_91h1r94kg21qt1jt20lxkc0000gn/T/music21/local-newCorpus.p.gz
bundles.py: WARNING: MetadataBundle Modification Time: 1590268167.182442
bundles.py: WARNING: Skipped 0 sources already in cache.
bundles.py: WARNING: updated 1 of 1 files; total errors: 0 ... last file: /Users/cuthbert/Desktop/nbval/README.md
/Users/cuthbert/git/music21base/music21/corpus/corpora.py: WARNING: cache: writing time: 0.153 md items: 0

/Users/cuthbert/git/music21base/music21/corpus/corpora.py: WARNING: cache: filename: /var/folders/1t/0_91h1r94kg21qt1jt20lxkc0000gn/T/music21/local-newCorpus.p.gz


True

We can see that our new *local* corpus is saved by checking for the names of
all saved *local* corpora using the corpus.manager list:

In [5]:
#_DOCS_SHOW corpus.manager.listLocalCorporaNames()
print("[None, 'funk', 'newCorpus', 'bach']") #_DOCS_HIDE

[None, 'funk', 'newCorpus', 'bach']


Finally, we can delete the *local* corpus we previously created like this:

In [6]:
aNewLocalCorpus.delete()
aNewLocalCorpus.existsInSettings

False

## Inspecting metadata bundle search results

Let's take a closer look at some search results:

In [7]:
bachBundle = corpus.corpora.CoreCorpus().search('bach', 'composer')
bachBundle

<music21.metadata.bundles.MetadataBundle {363 entries}>

In [8]:
bachBundle[0]

<music21.metadata.bundles.MetadataEntry 'bach_bwv10_7_mxl'>

In [9]:
bachBundle[0].sourcePath

PosixPath('bach/bwv10.7.mxl')

In [10]:
bachBundle[0].metadata

<music21.metadata.RichMetadata id=0x1107c8940>

In [11]:
bachBundle[0].metadata.all()

[('ambitus',
  "AmbitusShort(semitones=34, diatonic='m7', pitchLowest='G2', pitchHighest='F5')"),
 ('composer', 'J.S. Bach'),
 ('keySignatureFirst', '-2'),
 ('keySignatures', '[-2]'),
 ('movementName', 'bwv10.7.mxl'),
 ('noteCount', '214'),
 ('numberOfParts', '4'),
 ('pitchHighest', 'F5'),
 ('pitchLowest', 'G2'),
 ('quarterLength', '88.0'),
 ('sourcePath', 'bach/bwv10.7.mxl'),
 ('timeSignatureFirst', '4/4'),
 ('timeSignatures', "['4/4']")]

In [12]:
mdpl = bachBundle[0].metadata
mdpl.noteCount

214

In [13]:
bachAnalysis0 = bachBundle[0].parse()
bachAnalysis0.show()

## Manipulating multiple metadata bundles

Another useful feature of `music21`'s metadata bundles is that they can be
operated on as though they were sets, allowing you to union, intersect and
difference multiple metadata bundles, thereby creating more complex search
results:

In [14]:
corelliBundle = corpus.search('corelli', field='composer')
corelliBundle

<music21.metadata.bundles.MetadataBundle {1 entry}>

In [15]:
bachBundle.union(corelliBundle)

<music21.metadata.bundles.MetadataBundle {364 entries}>

Consult the API for class:`~music21.metadata.bundles.MetadataBundle` for a more
in depth look at how this works.

## Getting a metadata bundle

In music21, metadata is information *about* a score, such as its composer,
title, initial key signature or ambitus. A metadata *bundle* is a collection of
metadata pulled from an arbitrarily large group of different scores. Users can
search through metadata bundles to find scores with certain qualities, such as
all scores in a given corpus with a time signature of ``6/8``, or all scores
composed by Monteverdi.

There are a number of different ways to acquire a metadata bundle.  The easiest way
to get the metadataBundle for the core corpus is simply to download music21: we
include a pre-made metadataBundle (in ``corpus/metadataCache/core.json``) so
that this step is unnecessary for the core corpus unless you're contributing to
the project.  But you may want to create metadata bundles for your own local corpora.
Access the ``metadataBundle`` attribute of any ``Corpus`` instance to get its
corresponding metadata bundle:

In [16]:
coreCorpus = corpus.corpora.CoreCorpus()
coreCorpus.metadataBundle

<music21.metadata.bundles.MetadataBundle 'core': {15105 entries}>

Music21 also provides a handful of convenience methods for getting metadata
bundles associated with the *virtual*, *local* or *core* corpora:

In [17]:
coreBundle = corpus.corpora.CoreCorpus().metadataBundle
localBundle = corpus.corpora.LocalCorpus().metadataBundle
otherLocalBundle = corpus.corpora.LocalCorpus('blah').metadataBundle

But really advanced users can also make metadata bundles manually, by passing in the name of the
corpus you want the bundle to refer to, or, equivalently, an actual ``Corpus`` instance
itself:

In [18]:
coreBundle = metadata.bundles.MetadataBundle('core')
coreBundle = metadata.bundles.MetadataBundle(corpus.corpora.CoreCorpus())

However, you'll need to read the bundle's saved data from disk before you can
do anything useful with the bundle. Bundles don't read their associated JSON
files automatically when they're manually instantiated.

In [19]:
coreBundle

<music21.metadata.bundles.MetadataBundle 'core': {0 entries}>

In [20]:
coreBundle.read()

<music21.metadata.bundles.MetadataBundle 'core': {15105 entries}>

## Creating persistent metadata bundles

Metadata bundles can take a long time to create.  So it'd be nice if they could be written to and read from disk.  Unfortunately we never got around to...nah, just kidding.  Of course you can.  Just call `.write()` on one:

In [21]:
coreBundle = metadata.bundles.MetadataBundle('core')
coreBundle.read()

<music21.metadata.bundles.MetadataBundle 'core': {15105 entries}>

In [22]:
#_DOCS_SHOW coreBundle.write()

They can also be completely rebuilt, as you will want to do for local
corpora. To add information to a bundle, use the ``addFromPaths()`` method:

In [23]:
newBundle = metadata.bundles.MetadataBundle()
paths = corpus.corpora.CoreCorpus().search('corelli')
#_DOCS_SHOW failedPaths = newBundle.addFromPaths(paths)
failedPaths = [] #_DOCS_HIDE
failedPaths

[]

then call ``.write()`` to save to disk

In [24]:
#_DOCS_SHOW newBundle
print("<music21.metadata.bundles.MetadataBundle {1 entry}>") # did not actually run addFromPaths... #_DOCS_HIDE

<music21.metadata.bundles.MetadataBundle {1 entry}>


You can delete, rebuild and save a metadata bundle in one go with the
``rebuild()`` method:

In [25]:
localBundle = corpus.corpora.LocalCorpus().metadataBundle
#_DOCS_SHOW localBundle.rebuild()

The process of rebuilding will store the file as it goes (for safety) so at the end there is 
no need to call ``.write()``.

To delete a metadata bundle's cached-to-disk JSON file, use the ``delete()``
method:

In [26]:
#_DOCS_SHOW localBundle.delete()

Deleting a metadata bundle's JSON file won't empty the in-memory contents of
that bundle. For that, use ``clear()``:

In [27]:
localBundle.clear()